# Genome Assembly workbook - Mayo-Illinois Course 2024

## Test the environment

We need the following modules preloaded before we start this notebook! 

* `seqkit/2.6.1`
* `hifiasm/0.19.6-IGB-gcc-8.2.0`
* `Bandage/0.8.1`
* `gfatools/0.5-IGB-gcc-8.2.0`

This has been simplified thankfully.  These can be all loaded in using the single `Mayo/2024-IGB-gcc-8.2.0` module in the 'Softwares' tab (the hexagon-y looking tab on the left side).

We can then test this with the following cell, which should print a help menu.  We can then collapse it.

If it doesn't work and there is a bash kernel showing in the top right corner, you will need to shut the kernel down, load the modules using the 'Softwares' tab on the left tab, and then restart the kernel and rerun the command below

In [ ]:
seqkit -h

## Copy over the data

All good above?  Now we can copy over the data into the folder we are working in.

In [ ]:
cp -r /home/classroom/mayo/2024/Genome-Assembly/data/* . 

## Run seqkit stats on the raw sequence data

Let's check how much data we have in the four different data sets.

In [ ]:
seqkit stats --quiet dataset*.fastq.gz 

The overall stats above should indicate how much genome coverage there is per sample.  The genome size for this bacterium is known, about 1.7Mb.  For standard assemblies we try to aim for minimally 20-30x minimum, though we can assemble lower coverage with some caveats (as you'll see).  Even 30x might be a little low

We want to keep a record of the data, so let's rerun the above but use the `-T` option (fo creating tab-delimited output) and redirect the output to a file.  Notice that `.tsv` (tab-delimited) files in the left-side file browser can be opened by double-clicking into a table format, which can be pretty handy.

In [ ]:
seqkit stats -T --quiet dataset*.fastq.gz > seqkit-reads.tsv

## **Question:** 

What is the estimated genome coverage for each data set above?  How would you calculate this?

## Run assembly 1

Now we are going to run our assemblies.  The first will take about 7-8 minutes, maybe more if everyone is running these all at once.

In [ ]:
mkdir -p dataset1
time hifiasm -o dataset1/full.asm --n-hap 1 -l0 -t $SLURM_NTASKS dataset1.fastq.gz 2> dataset1/full.log

## Run assembly 2

Assembly 2 has half the coverage of the first.

In [ ]:
mkdir -p dataset2
time hifiasm -o dataset2/half.asm --n-hap 1 -l0 -t $SLURM_NTASKS dataset2.fastq.gz 2> dataset2/half.log

## Run assembly 3

Assembly 3 has 1/4 the coverage of the first.

In [ ]:
mkdir -p dataset3
time hifiasm -o dataset3/quarter.asm --n-hap 1 -l0 -t $SLURM_NTASKS dataset3.fastq.gz 2> dataset3/quarter.log

## Run assembly 4

Assembly 4 has 1/10 the coverage of the first, so the overall coverage is pretty sparse.

In [ ]:
mkdir -p dataset4
time hifiasm -o dataset4/sparse.asm --n-hap 1 -l0 -t $SLURM_NTASKS dataset4.fastq.gz 2> dataset4/sparse.log

## **Question** 

Can you see any obvious pattern from the times the assemblies took?

## Convert sequence formats

We commonly need to convert from one bioinformatics format to another.  In this case, hifiasm produces GFA files, which are useful but isn't a format that most downstream tools use, which is a simpler format called FASTA.  Here, we do a simple conversion of the primary contig files from GFA to FASTA using a tool called `gfatools` (appropriately enought). Some of these tools are multiuse and have subcommands; in this case the subcommand to convert from GFA to FASTA is `gfa3fa`.

In [ ]:
gfatools gfa2fa dataset1/full.asm.bp.p_ctg.gfa > dataset1/full.asm.bp.p_ctg.fasta 2> /dev/null
gfatools gfa2fa dataset2/half.asm.bp.p_ctg.gfa > dataset2/half.asm.bp.p_ctg.fasta 2> /dev/null
gfatools gfa2fa dataset3/quarter.asm.bp.p_ctg.gfa > dataset3/quarter.asm.bp.p_ctg.fasta 2> /dev/null
gfatools gfa2fa dataset4/sparse.asm.bp.p_ctg.gfa > dataset3/sparse.asm.bp.p_ctg.fasta 2> /dev/null

In [ ]:
ls -l dataset*/*.fasta

## Generate assembly stats

Now we want to see some stats for the sequences.  Here we can use `seqkit` again, but we also want the N50 value.  We can get this using `seqkit stats` by requesting full stats.  Let's first save it to a file (the extended stats make the raw output hard to read).

In [ ]:
seqkit stats -T --quiet -a dataset*/*.fasta > seqkit-assembly.stats.tsv

Now we can open this file separately.  In the file browser, double-click to open the file.

## Visualize data using Bandage

We can visualize the assemblies using the tool Bandage.  We have an optional section in the lab for using Bandage using the GUI, but you can also use Bandage for generating simple graphs of the assemblies.  Let's look at the first one (dataset1).  

The following command will generate a PNG file

In [ ]:
Bandage image dataset1/full.asm.bp.p_ctg.noseq.gfa dataset1/full.png

This command can then have the image (PNG or JPEG) displayed in the notebook:

In [ ]:
cat dataset1/full.png | display

This should be a closed circular genome (hopefully this is what you get.

We can now look at the other assemblies.  For dataset2 ('half'):

In [ ]:
Bandage image dataset2/half.asm.bp.p_ctg.noseq.gfa dataset2/half.png
cat dataset2/half.png | display

Not quite closed, but one contig. Not too bad!

And now for dataset3 ('sparse').  

In [ ]:
Bandage image dataset3/quarter.asm.bp.p_ctg.noseq.gfa dataset3/quarter.png
cat dataset3/quarter.png | display

A bit more fragmented; we're starting to see effects of lower genome coverage.  And now for the worst one, assembly 4:

In [ ]:
Bandage image dataset4/sparse.asm.bp.p_ctg.noseq.gfa dataset4/sparse.png
cat dataset4/sparse.png | display

Significantly more fragmented!

We can also look at the other GFA files.  We have been looking at primary _contig_ graphs.  A _unitig_ graph includes all connections in the assembly within reason (e.g., a certain level of read coverage and base quality of he reads). In other words these can include regions that show some degree of variation, though it may be low coverage and possibly erroneous.  These are generally cleaned up in the primary assembly if the number of haplotypes expected is less than 2 (like this case). 

In [ ]:
Bandage image dataset1/full.asm.bp.p_utg.noseq.gfa dataset1/full.unitig.png
cat dataset1/full.unitig.png | display

Here we can see there are some more complex regions even for a bacterial assembly.  Since these are removed in the primary contig assembly, the other paths were likely pruned (removed) based on poor read evidence.